In this notebook we normalize the data from the INE mobility study

In [1]:
import pandas as pd

import numpy as np

import os
import datetime
from datetime import date, timedelta
from tqdm import tqdm_notebook

data_path = os.path.join('/','home','fterroso','data','Spain-INE-mobility-study-2020','muestra1')

In [32]:
e_date = datetime.datetime.strptime('2020-11-30', '%Y-%m-%d')
i_date = datetime.datetime.strptime('2020-02-21', '%Y-%m-%d')

# General functions

In [3]:
def generate_days_fn(init_date, final_date):
    
    delta = final_date - init_date       # as timedelta

    target_days = []
    for i in range(delta.days + 1):
        day = init_date + timedelta(days=i)
        target_days.append(day)
    
    return target_days

In [4]:
def read_INE_trips_date_fn(date_, trips_type='all'):
    date_str= date_.strftime('%Y%m%d')
    df_date= pd.read_csv(os.path.join(data_path, f'{date_str}_maestra_1_mitma_distrito.txt'), 
                             sep='|',dtype={'origen':str, 'destino':str,'fecha':str, 'periodo':str})
    
    if trips_type=='inter':
        df_date= df_date[df_date['origen']!=df_date['destino']] #only keep trips between areas
    elif trips_type=='intra':
        df_date= df_date[df_date['origen']==df_date['destino']] #only keep trips within the areas
    
    #convert period column to a two-digit string
    df_date['periodo'] = df_date['periodo'].apply(lambda x: x.zfill(2))
    df_date= df_date.fillna(0) # set nan as 0
    return df_date


In [35]:
def generate_daily_time_series_fn(from_date, to_date, trips_type='all'):
     
    time_series = []
    target_days = generate_days_fn(from_date, to_date)
    for date in tqdm_notebook(target_days):
        df = read_INE_trips_date_fn(date, trips_type)
        df= df.drop(columns=['edad', 'viajes_km','residencia','periodo'])
        group_by_df= df.groupby(['fecha','origen']).sum('viajes')
        group_by_df= group_by_df.reset_index()
        total_trips_final = group_by_df.pivot_table(values='viajes',index='origen',columns='fecha')
        time_series.append(total_trips_final)
    return pd.concat(time_series, axis=1)

In [36]:
def generate_hourly_time_series_fn(from_date, to_date, trips_type='all'):
     
    time_series = []
    target_days = generate_days_fn(from_date, to_date)
    for date in tqdm_notebook(target_days):
        df = read_INE_trips_date_fn(date, trips_type)
        df= df.drop(columns=['edad', 'viajes_km','residencia'])
        group_by_df= df.groupby(['fecha','origen','periodo']).sum('viajes')
        group_by_df= group_by_df.reset_index()
        group_by_df['fecha_periodo'] = group_by_df['fecha']+'_'+ group_by_df['periodo'].astype(str)
        group_by_df= group_by_df.drop(columns='fecha periodo'.split())
        total_trips_final = group_by_df.pivot_table(values='viajes',index='origen',columns='fecha_periodo')
        time_series.append(total_trips_final)
    return pd.concat(time_series, axis=1)

In [37]:
def generate_OD_matrix_from_date_fn(date_):
    df = read_INE_trips_date_fn(date_)
    od= pd.pivot_table(df, columns='destino', index='origen', values='viajes', aggfunc=np.sum)
    od = od.fillna(0.0)
    od['total'] = od.sum(axis=1)
    od.loc['total']= od.sum(axis=0)
    return od

## Generate OD matrices

In [40]:
    target_days = generate_days_fn(i_date, e_date)
    for date in tqdm_notebook(target_days):
        od= generate_OD_matrix_from_date_fn(date)
        od.to_csv(os.path.join('data', 'OD', 'od_matrix_{}'.format(date.strftime("%Y-%m-%d"))))

In [41]:
od.head()

destino,01001_AM,01002,01010_AM,01031_AM,01036,01043_AM,01047_AM,01051,01058_AM,0105901,...,5100105,5100106,5200101_AD,5200102,5200104,5200105,5200106,5200107,5200108,total
origen,,,,,,,,,,,,,,,,,,,,,
01001_AM,3008.834,19.483,0.000,43.957,23.780,442.603,30.877,2934.326,573.934,1115.337,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19977.788
01002,60.619,7061.001,2269.831,0.000,2366.466,3.819,43.058,43.058,7.566,29.054,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17391.825
01010_AM,0.000,2370.761,1370.828,39.298,1572.767,0.000,32.448,0.000,16.224,32.509,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9260.783
01031_AM,18.187,33.213,39.298,2623.549,21.703,1166.576,337.645,30.595,27.974,126.036,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12484.364
01036,42.375,2291.997,1555.562,7.508,11989.705,0.000,11.277,0.000,33.239,82.099,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25658.845


In [42]:
od.shape

(2832, 2832)

In [43]:
od.tail()

destino,01001_AM,01002,01010_AM,01031_AM,01036,01043_AM,01047_AM,01051,01058_AM,0105901,...,5100105,5100106,5200101_AD,5200102,5200104,5200105,5200106,5200107,5200108,total
origen,,,,,,,,,,,,,,,,,,,,,
5200105,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,1826.500,2230.327,2845.315,4429.933,4605.947,4501.340,12156.428,3.269418e+04
5200106,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,1699.265,1072.769,3615.129,4694.895,948.867,5338.590,7951.281,2.538286e+04
5200107,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,2645.589,3369.918,3696.834,4578.057,4699.148,3107.262,14389.167,3.653552e+04
5200108,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,4738.175,5177.529,10034.044,12648.722,7749.776,14574.824,27901.035,8.297320e+04
total,20062.724,17334.352,8706.941,12501.944,26226.718,13148.431,8646.692,7717.527,17604.293,91304.113,...,29080.041,44640.408,13156.095,15000.374,23840.879,33914.844,24993.656,36672.330,82247.987,1.157682e+08


## Generate time series with all the trips (inter and intra) (hour granularity)

In [223]:
df = generate_hourly_time_series_fn(datetime.datetime.strptime('2020-02-21', '%Y-%m-%d'), 
                                    datetime.datetime.strptime('2020-11-30', '%Y-%m-%d'))

In [224]:
df.head()

fecha_periodo,20200221_00,20200221_01,20200221_02,20200221_03,20200221_04,20200221_05,20200221_06,20200221_07,20200221_08,20200221_09,...,20201130_14,20201130_15,20201130_16,20201130_17,20201130_18,20201130_19,20201130_20,20201130_21,20201130_22,20201130_23
01001_AM,351.569,151.978,170.314,304.908,366.064,612.043,792.522,1347.836,1078.382,1083.844,...,1521.115,1223.343,1269.692,1264.783,826.047,742.421,673.619,848.775,348.361,338.776
01002,307.557,299.509,234.590,119.108,402.865,776.728,900.781,1033.930,894.766,1088.407,...,1199.787,1037.809,970.586,1289.262,1075.365,780.131,715.145,732.227,452.936,249.190
01010_AM,117.676,74.781,73.922,33.134,116.510,430.138,476.429,630.298,552.545,517.019,...,607.014,656.945,582.302,536.971,433.186,305.935,302.467,320.358,200.024,79.813
01031_AM,124.948,107.608,110.947,82.437,150.341,222.748,541.171,915.834,773.469,907.130,...,1292.460,857.127,823.528,784.795,693.835,446.964,355.163,321.990,166.776,143.830
01036,515.327,232.868,372.743,436.898,586.419,1066.202,1589.005,1448.471,1420.790,1687.426,...,1702.522,1385.129,1420.070,1458.191,1512.568,1270.918,1192.500,963.190,748.999,307.763


In [225]:
df.tail()

fecha_periodo,20200221_00,20200221_01,20200221_02,20200221_03,20200221_04,20200221_05,20200221_06,20200221_07,20200221_08,20200221_09,...,20201130_14,20201130_15,20201130_16,20201130_17,20201130_18,20201130_19,20201130_20,20201130_21,20201130_22,20201130_23
38013_AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
05220_AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40149_AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31092_AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,334.077,487.17,401.15,465.95,140.652,277.039,192.494,9.76,84.61,NaN
20903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [226]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2844 entries, 01001_AM to 20903
Columns: 6815 entries, 20200221_00 to 20201130_23
dtypes: float64(6815)
memory usage: 147.9+ MB


In [227]:
df.to_csv(os.path.join('data','trips_time_series_all.csv'))

## Generate time series with inner trips (hour granularity)

In [228]:
df_intra = generate_hourly_time_series_fn(datetime.datetime.strptime('2020-02-21', '%Y-%m-%d'),
                                          datetime.datetime.strptime('2020-11-30', '%Y-%m-%d'), 
                                          trips_type='intra')

In [ ]:
df_intra.head()

fecha_periodo,20200221_00,20200221_01,20200221_02,20200221_03,20200221_04,20200221_05,20200221_06,20200221_07,20200221_08,20200221_09,...,20201130_14,20201130_15,20201130_16,20201130_17,20201130_18,20201130_19,20201130_20,20201130_21,20201130_22,20201130_23
01001_AM,59.915,22.328,46.492,24.098,83.646,93.864,56.448,94.836,96.838,190.334,...,213.877,197.260,195.845,117.577,175.307,94.397,110.305,113.574,89.900,108.185
01002,115.028,149.517,184.043,46.011,194.751,159.942,168.706,254.343,256.654,353.640,...,498.194,439.049,354.764,462.380,494.522,401.619,333.450,324.661,328.207,142.446
01010_AM,24.950,23.959,36.027,13.548,24.950,33.607,19.249,38.972,35.626,46.011,...,111.554,46.824,22.506,60.091,78.379,90.224,85.874,42.784,80.194,27.482
01031_AM,27.274,42.707,18.797,19.768,41.285,14.286,71.247,224.269,127.676,117.224,...,206.847,228.601,140.950,202.339,89.877,102.807,68.622,52.335,7.333,25.801
01036,341.762,114.641,167.959,217.580,176.934,287.099,497.392,478.965,575.706,811.725,...,631.980,569.517,738.954,711.274,807.784,857.207,750.512,387.753,429.239,238.795


In [ ]:
df_intra.tail()

fecha_periodo,20200221_00,20200221_01,20200221_02,20200221_03,20200221_04,20200221_05,20200221_06,20200221_07,20200221_08,20200221_09,...,20201130_14,20201130_15,20201130_16,20201130_17,20201130_18,20201130_19,20201130_20,20201130_21,20201130_22,20201130_23
41101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_intra.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2836 entries, 01001_AM to 30010
Columns: 6815 entries, 20200221_00 to 20201130_23
dtypes: float64(6815)
memory usage: 147.5+ MB


In [ ]:
df_intra.to_csv(os.path.join('data','trips_time_series_all_intra.csv'))

## Generate time series inter trips (hour-granularity)

In [ ]:
df_inter = generate_hourly_time_series_fn(datetime.datetime.strptime('2020-02-21', '%Y-%m-%d'),
                                          datetime.datetime.strptime('2020-11-30', '%Y-%m-%d'),
                                          trips_type='inter')

In [ ]:
df_inter.head()

In [ ]:
df_inter.tail()

In [ ]:
df_inter.to_csv(os.path.join('data','trips_time_series_all_inter.csv'))

--------------------------------

# Generate daily time series

### All trips

In [ ]:
df_all = generate_daily_time_series_fn(datetime.datetime.strptime('2020-02-21', '%Y-%m-%d'),
                                       datetime.datetime.strptime('2020-11-30', '%Y-%m-%d'))

In [ ]:
df_all.head()

In [ ]:
df_all.tail()

In [ ]:
df_all.to_csv(os.path.join('data','trips_time_series_all_day.csv'))

### Intra trips

In [7]:
df_intra = generate_daily_time_series_fn(datetime.datetime.strptime('2020-02-21', '%Y-%m-%d'),
                                         datetime.datetime.strptime('2020-11-30', '%Y-%m-%d'),
                                         trips_type='intra')

In [8]:
df_intra.head()

fecha,20200221,20200222,20200223,20200224,20200225,20200226,20200227,20200228,20200229,20200301,...,20201121,20201122,20201123,20201124,20201125,20201126,20201127,20201128,20201129,20201130
01001_AM,3060.845,3025.964,3025.544,3138.833,3153.149,2961.794,2820.691,2951.962,2394.570,2434.055,...,2970.056,2918.984,2996.803,3684.630,3251.133,3573.987,3001.207,4651.793,3486.482,3008.834
01002,7153.169,6776.737,5552.504,7630.354,7538.326,7863.609,7139.375,8384.672,6238.427,6605.095,...,6067.722,7856.853,4761.303,8919.366,7980.826,7938.313,7763.090,5608.531,5666.668,7061.001
01010_AM,1646.253,1535.491,1078.463,1392.290,1304.553,1160.057,1548.302,1534.356,1234.247,1263.346,...,1068.980,1108.179,1189.631,2117.145,1457.238,1398.737,1085.345,1144.112,1162.197,1370.828
01031_AM,3200.834,2801.484,2450.735,3057.690,3552.169,2659.732,3325.969,3273.036,3294.249,2857.638,...,1965.434,1577.658,2429.796,2152.553,2493.095,2583.649,2462.348,1705.231,1843.687,2623.549
01036,16073.506,15709.096,13914.451,13864.802,14559.735,15119.090,14922.458,15215.656,13291.284,12133.353,...,13154.262,12200.504,14283.554,13046.072,14750.203,14407.372,12367.775,12040.679,9684.807,11989.705


In [9]:
df_intra.tail()

fecha,20200221,20200222,20200223,20200224,20200225,20200226,20200227,20200228,20200229,20200301,...,20201121,20201122,20201123,20201124,20201125,20201126,20201127,20201128,20201129,20201130
41101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_intra.to_csv(os.path.join('data','trips_time_series_all_intra_day.csv'))

### Inter trips

In [11]:
df_inter = generate_daily_time_series_fn(datetime.datetime.strptime('2020-02-21', '%Y-%m-%d'),
                                         datetime.datetime.strptime('2020-11-30', '%Y-%m-%d'),
                                         trips_type='inter')

In [12]:
df_inter.head()

fecha,20200221,20200222,20200223,20200224,20200225,20200226,20200227,20200228,20200229,20200301,...,20201121,20201122,20201123,20201124,20201125,20201126,20201127,20201128,20201129,20201130
01001_AM,17520.424,15372.711,14916.241,17965.837,16967.247,17207.530,17689.596,19447.444,14921.379,13583.044,...,12033.831,10981.673,15817.721,17126.542,15856.056,17361.117,16483.380,12067.261,11355.599,16968.954
01002,12801.148,9749.310,7710.079,11880.968,11979.052,12493.285,12537.988,12466.178,9806.607,7766.773,...,5777.141,5790.425,9974.630,10512.281,10629.208,10278.895,9873.569,6244.468,4821.780,10330.824
01010_AM,10088.279,7666.813,6495.168,8439.486,8678.470,9356.337,8968.290,9736.112,8083.501,6808.596,...,5381.512,5436.622,7827.202,8427.585,7575.841,8134.337,7963.381,5699.995,4091.578,7889.955
01031_AM,14629.763,12657.978,11687.316,13212.275,13581.972,14223.670,14247.865,15416.497,15339.991,12743.615,...,6339.804,6036.619,11280.772,10418.808,10973.507,9754.900,9583.168,6541.193,5515.221,9860.815
01036,19140.246,11235.947,11305.559,16814.806,17020.882,17854.522,18688.421,18235.443,13501.545,9756.308,...,7224.326,4893.469,13695.627,14961.569,15054.527,15369.381,14577.721,7624.625,4899.865,13669.140


In [13]:
df_inter.tail()

fecha,20200221,20200222,20200223,20200224,20200225,20200226,20200227,20200228,20200229,20200301,...,20201121,20201122,20201123,20201124,20201125,20201126,20201127,20201128,20201129,20201130
38013_AM,NaN,NaN,NaN,NaN,NaN,3762.972,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
05220_AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,9053.431,NaN,NaN,NaN
40149_AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31092_AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3980.047,5408.309,2592.019,2302.188,2391.693,2327.155,2367.877,2714.1,5375.049,2894.267
20903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df_inter.to_csv(os.path.join('data','trips_time_series_all_inter_day.csv'))

# Test code

In [5]:
df = read_INE_trips_date_fn(datetime.datetime.strptime('2020-02-29', '%Y-%m-%d'))

In [6]:
df['distancia'].unique()

array(['002-005', '005-010', '010-050', '0005-002', '050-100', '100+'],
      dtype=object)

In [16]:
df['periodo'] = df['periodo'].apply(lambda x: x.zfill(2))

In [17]:
df= df.drop(columns=['edad', 'viajes_km','residencia'])

In [18]:
df.head()

,fecha,origen,destino,actividad_origen,actividad_destino,periodo,distancia,viajes
0,20200229,01001_AM,01001_AM,casa,otros,00,002-005,7.067
1,20200229,01001_AM,01001_AM,casa,otros,00,005-010,8.807
2,20200229,01001_AM,01001_AM,casa,otros,01,002-005,5.795
3,20200229,01001_AM,01001_AM,casa,otros,02,005-010,5.795
4,20200229,01001_AM,01001_AM,casa,otros,02,010-050,4.404


In [19]:
df[(df['fecha']=='20201129')& (df['origen']=='01001_AM') & (df['periodo']=='00')]

,fecha,origen,destino,actividad_origen,actividad_destino,periodo,distancia,viajes


In [20]:
df[(df['fecha']=='20201129')& (df['origen']=='01001_AM') & (df['periodo']=='01')]['viajes'].sum()

0.0

In [21]:
total_trips= df.groupby(['fecha','origen']).sum('viajes')

In [22]:
total_trips

viajes
fecha    origen             
20200229 01001_AM  17315.949
         01002     16045.034
         01010_AM   9317.748
         01031_AM  18634.240
         01036     26792.829
...                      ...
         5200104   23837.682
         5200105   33171.496
         5200106   20541.173
         5200107   38137.500
         5200108   80042.550

[2834 rows x 1 columns]

In [23]:
total_trips= total_trips.reset_index()

In [24]:
total_trips

,fecha,origen,viajes
0,20200229,01001_AM,17315.949
1,20200229,01002,16045.034
2,20200229,01010_AM,9317.748
3,20200229,01031_AM,18634.240
4,20200229,01036,26792.829
...,...,...,...
2829,20200229,5200104,23837.682
2830,20200229,5200105,33171.496
2831,20200229,5200106,20541.173
2832,20200229,5200107,38137.500


In [25]:
'10'.zfill(2)

'10'

In [26]:
total_trips['fecha_periodo'] = total_trips['fecha']+'_'+ total_trips['periodo'].astype(str)

KeyError: 'periodo'

In [ ]:
total_trips.head()

In [ ]:
total_trips= total_trips.drop(columns='fecha periodo'.split())

In [ ]:
total_trips.head()

In [ ]:
total_trips_final = total_trips.pivot_table(values='viajes',index='origen',columns='fecha_periodo')

In [ ]:
total_trips_final.head()

In [ ]:
total_trips_final.tail()

## Distributed processing (global stats)

In [ ]:
import dask.dataframe as dd
from dask.distributed import Client, progress
client = Client(threads_per_worker=4, n_workers=10)
client

In [ ]:
df = dd.read_csv(os.path.join(data_path, '2020*.txt'), sep='|',dtype={'origen':str, 'destino':str})

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

We drop column 'edad' as it only contains nan values

In [ ]:
df_no_age = df.drop(columns='edad')

In [ ]:
df_no_age.head()

In [ ]:
df_no_age['actividad_destino'].unique().compute()

In [ ]:
df_no_age['actividad_origen'].unique().compute()

In [ ]:
df_no_age['distancia'].unique().compute()

In [ ]:
df_no_age['residencia'].unique().compute()

In [ ]:
group_by_dist_df= df_no_age.groupby('distancia')

In [ ]:
trips_dist = group_by_dist_df.size().compute()

In [ ]:
trips_dist_df = trips_dist.to_frame()

In [ ]:
trips_dist_df.head()

In [ ]:
trips_dist_df.plot(kind='bar')

In [ ]:
#df_no_age.to_csv(os.path.join('/','tmp','clean_ine_trips-*.csv'))  

In [ ]:
print("That's all folks")